Only the first 10k rows from the dataset are considered due to the system's handling capacity

In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import string
import re
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/IMDB Dataset.csv") # reading the csv file
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data = data.iloc[0:10000] # taking the first 10k rows from the entire dataset, as a new dataframe
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
##function to preprocess the data.
def preprocess_data(review):
    
     
    review = re.sub(r'</?\s*br\s*/?>', '', review)## to replace linebreak with a space
    review = re.sub(r'([^\w\s])\1+', r'\1', review)### remove punctuations occuring multiple times together. Eg,'????','!!!!','......'
    review = review.lower()  # Lower casing all the characters
    review = re.sub(r'[^\x00-\x7F]+', '',review) # Replace all the characters except alphabets from the text
    review = re.sub(r'\'', '', review)
    return review

In [ ]:
data['review'] = data['review'].apply(lambda x: preprocess_data(x)) ##calling the preprocessing function through a lamda function
data['review']

0       one of the other reviewers has mentioned that ...
1       a wonderful little production. the filming tec...
2       i thought this was a wonderful way to spend ti...
3       basically theres a family where a little boy (...
4       petter matteis "love in the time of money" is ...
                              ...                        
9995    fun, entertaining movie about wwii german spy ...
9996    give me a break. how can anyone say that this ...
9997    this movie is a bad movie. but after watching ...
9998    this is a movie that was probably made to ente...
9999    smashing film about film-making. shows the int...
Name: review, Length: 10000, dtype: object

In [ ]:

# Import label encoder
from sklearn import preprocessing
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout  

label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'sentiment'
sentiment = label_encoder.fit_transform(data['sentiment'])
##label coded sentiment to one hot encoding
sentiment =tf.keras.utils.to_categorical(sentiment)


In [ ]:
sentiment

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [ ]:
review=data['review'].tolist() # converting the 'review' to a list before tokenizing

In [ ]:
vocab_size=4000 #initialising the number of unique words as vocab_size and the number of embedding input feature dimension
embedding_dim=100

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# create tokenizer
tokenizer = Tokenizer(num_words = vocab_size,oov_token="<OOV>")

# fit tokenizer on reviews
tokenizer.fit_on_texts(review)

# convert text to sequences of integers
sequences = tokenizer.texts_to_sequences(review)

# get max length of a sequence
max_length = max(len(seq) for seq in sequences)

# perform zero padding in all the sequences in order to match the max length of the sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequences = pad_sequences(sequences, padding='pre', maxlen=max_length)

print(padded_sequences)

[[   0    0    0 ...  122 3817  484]

 [   0    0    0 ... 2158   68  214]

 [   0    0    0 ...   63   16  340]

 ...

 [   0    0    0 ...   15  330  153]

 [   0    0    0 ...   49   17   70]

 [   0    0    0 ...    7  983 2682]]


In [ ]:
X=padded_sequences

In [ ]:
import numpy as np

y=np.array(sentiment) # converting the one hot encoded sentiment to array to match the type of padded sequence

In [ ]:
X_Train,X_Test,Y_Train,Y_Test=train_test_split(X,y,test_size=0.2,random_state=42) # train test split as 80-20

X_Val,X_Test,Y_Val,Y_Test=train_test_split(X_Test,Y_Test,test_size=0.5,random_state=42) #test val split as 10-10

In [ ]:
#printing the length of train,test,val data
print('X-train length',len(X_Train))
print('Y-train length',len(Y_Train))
print('X-testlength',len(X_Test))
print('Y-test length',len(Y_Test))
print('X-val length',len(X_Val))
print('Y-val length',len(Y_Val))


X-train length 8000

Y-train length 8000

X-testlength 1000

Y-test length 1000

X-val length 1000

Y-val length 1000


## RNN

In [ ]:

# Define the model architecture with one embedding, RNN and fully connected layer
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(SimpleRNN(units=256, dropout = 0.2, activation='sigmoid')) # 256 is the number of neurons in the layer
model.add(Dense(units=2, activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])## compiling the model with the optimizer,lossfunction 

In [ ]:
model.fit(X_Train,Y_Train,validation_data=(X_Val,Y_Val), epochs=10, batch_size=128)#model fitting

Epoch 1/10

63/63 [==============================] - 167s 3s/step - loss: 0.7021 - accuracy: 0.5124 - val_loss: 0.6798 - val_accuracy: 0.5500

Epoch 2/10

63/63 [==============================] - 150s 2s/step - loss: 0.6465 - accuracy: 0.6376 - val_loss: 0.6475 - val_accuracy: 0.6110

Epoch 3/10

63/63 [==============================] - 149s 2s/step - loss: 0.5728 - accuracy: 0.6964 - val_loss: 0.6355 - val_accuracy: 0.6480

Epoch 4/10

63/63 [==============================] - 149s 2s/step - loss: 0.4954 - accuracy: 0.7540 - val_loss: 0.6326 - val_accuracy: 0.6570

Epoch 5/10

63/63 [==============================] - 148s 2s/step - loss: 0.4354 - accuracy: 0.7922 - val_loss: 0.7357 - val_accuracy: 0.6700

Epoch 6/10

63/63 [==============================] - 149s 2s/step - loss: 0.3784 - accuracy: 0.8275 - val_loss: 0.7142 - val_accuracy: 0.7020

Epoch 7/10

63/63 [==============================] - 149s 2s/step - loss: 0.3166 - accuracy: 0.8636 - val_loss: 0.6312 - val_accuracy: 0.7160


In [ ]:
loss, accuracy = model.evaluate(X_Test, Y_Test)##testing the model
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

32/32 [==============================] - 5s 169ms/step - loss: 0.6556 - accuracy: 0.7500

Test Loss: 0.6556399464607239

Test Accuracy: 0.75


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Get the predicted labels
y_pred = model.predict(X_Test)
y_pred = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded labels back to integers
y_true = np.argmax(Y_Test, axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.7573385518590998

Recall: 0.7543859649122807

F1-score: 0.7558593750000001


## LSTM

In [ ]:
from tensorflow.keras.layers import LSTM
# Define the model architecture
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=4000, output_dim=100, input_length=max_length))
lstm_model.add(LSTM(100))
lstm_model.add(Dropout(0.2))

lstm_model.add(Dense(units=2, activation='sigmoid'))




In [ ]:
# Compile the model
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
lstm_model.fit(X_Train,Y_Train,validation_data=(X_Val,Y_Val), epochs=10, batch_size=128)

Epoch 1/10

63/63 [==============================] - 424s 7s/step - loss: 0.6439 - accuracy: 0.6248 - val_loss: 0.5740 - val_accuracy: 0.7370

Epoch 2/10

63/63 [==============================] - 367s 6s/step - loss: 0.4325 - accuracy: 0.8173 - val_loss: 0.3797 - val_accuracy: 0.8400

Epoch 3/10

63/63 [==============================] - 362s 6s/step - loss: 0.2392 - accuracy: 0.9039 - val_loss: 0.3398 - val_accuracy: 0.8760

Epoch 4/10

63/63 [==============================] - 363s 6s/step - loss: 0.1944 - accuracy: 0.9265 - val_loss: 0.3809 - val_accuracy: 0.8440

Epoch 5/10

63/63 [==============================] - 365s 6s/step - loss: 0.1517 - accuracy: 0.9457 - val_loss: 0.3961 - val_accuracy: 0.8660

Epoch 6/10

63/63 [==============================] - 362s 6s/step - loss: 0.0955 - accuracy: 0.9711 - val_loss: 0.4579 - val_accuracy: 0.8570

Epoch 7/10

60/63 [===========================>..] - ETA: 16s - loss: 0.0616 - accuracy: 0.9836

In [ ]:
loss, accuracy = lstm_model.evaluate(X_Test, Y_Test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

32/32 [==============================] - 1s 22ms/step - loss: 0.5513 - accuracy: 0.8260

Test Loss: 0.5513419508934021

Test Accuracy: 0.8259999752044678


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Get the predicted labels
y_pred = lstm_model.predict(X_Test)
y_pred = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded labels back to integers
y_true = np.argmax(Y_Test, axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.8480492813141683

Recall: 0.8050682261208577

F1-score: 0.8260000000000001


## GRU

In [ ]:
from tensorflow.keras.layers import  GRU
# Define the model architecture
gru_model = Sequential()
gru_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
gru_model.add(GRU(256))
gru_model.add(Dense(units=2, activation='sigmoid'))

# Compile the model
gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
gru_model.fit(X_Train,Y_Train,validation_data=(X_Val,Y_Val), epochs=10, batch_size=128)

Epoch 1/10

63/63 [==============================] - 15s 190ms/step - loss: 0.6726 - accuracy: 0.6064 - val_loss: 0.5894 - val_accuracy: 0.7070

Epoch 2/10

63/63 [==============================] - 12s 186ms/step - loss: 0.4923 - accuracy: 0.7675 - val_loss: 0.5098 - val_accuracy: 0.7410

Epoch 3/10

63/63 [==============================] - 12s 188ms/step - loss: 0.3516 - accuracy: 0.8504 - val_loss: 0.4169 - val_accuracy: 0.8310

Epoch 4/10

63/63 [==============================] - 12s 192ms/step - loss: 0.2337 - accuracy: 0.9060 - val_loss: 0.4167 - val_accuracy: 0.8470

Epoch 5/10

63/63 [==============================] - 12s 192ms/step - loss: 0.1737 - accuracy: 0.9355 - val_loss: 0.4539 - val_accuracy: 0.8380

Epoch 6/10

63/63 [==============================] - 12s 190ms/step - loss: 0.1312 - accuracy: 0.9548 - val_loss: 0.5128 - val_accuracy: 0.8380

Epoch 7/10

63/63 [==============================] - 12s 191ms/step - loss: 0.0899 - accuracy: 0.9709 - val_loss: 0.5966 - val_acc

In [ ]:
loss, accuracy = gru_model.evaluate(X_Test, Y_Test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

32/32 [==============================] - 1s 33ms/step - loss: 0.8317 - accuracy: 0.8140

Test Loss: 0.8316957950592041

Test Accuracy: 0.8140000104904175


In [ ]:
# Get the predicted labels
y_pred = gru_model.predict(X_Test)
y_pred = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded labels back to integers
y_true = np.argmax(Y_Test, axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.8811188811188811

Recall: 0.7368421052631579

F1-score: 0.8025477707006369


## BidirectionalRNN

In [ ]:
from tensorflow.keras.layers import Bidirectional
hidden_dims=64
brnn_model = Sequential()
brnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
brnn_model.add(Dropout(0.2))
brnn_model.add(Bidirectional(SimpleRNN(100)))

brnn_model.add(Dense(2, activation='sigmoid'))

In [ ]:
# Compile the model
brnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
brnn_model.fit(X_Train, Y_Train, validation_data=(X_Val, Y_Val), batch_size = 128, epochs=10)

Epoch 1/10

63/63 [==============================] - 238s 4s/step - loss: 0.7012 - accuracy: 0.5034 - val_loss: 0.6896 - val_accuracy: 0.5270

Epoch 2/10

63/63 [==============================] - 223s 4s/step - loss: 0.6859 - accuracy: 0.5681 - val_loss: 0.6865 - val_accuracy: 0.5320

Epoch 3/10

63/63 [==============================] - 229s 4s/step - loss: 0.6696 - accuracy: 0.6097 - val_loss: 0.6639 - val_accuracy: 0.6100

Epoch 4/10

63/63 [==============================] - 225s 4s/step - loss: 0.6090 - accuracy: 0.7090 - val_loss: 0.5965 - val_accuracy: 0.6770

Epoch 5/10

63/63 [==============================] - 216s 3s/step - loss: 0.4662 - accuracy: 0.7890 - val_loss: 0.5223 - val_accuracy: 0.7390

Epoch 6/10

63/63 [==============================] - 221s 4s/step - loss: 0.3340 - accuracy: 0.8576 - val_loss: 0.4313 - val_accuracy: 0.8120

Epoch 7/10

63/63 [==============================] - 220s 3s/step - loss: 0.3170 - accuracy: 0.8670 - val_loss: 0.5740 - val_accuracy: 0.7370


In [ ]:
brnn_model.summary()

Model: "sequential"

_________________________________________________________________

Layer (type)                 Output Shape              Param #   


module_wrapper (ModuleWrappe (None, 1806, 100)         400000    

_________________________________________________________________

dropout (Dropout)            (None, 1806, 100)         0         

_________________________________________________________________

module_wrapper_1 (ModuleWrap (None, 200)               40200     

_________________________________________________________________

dense (Dense)                (None, 2)                 402       


Total params: 440,602

Trainable params: 440,602

Non-trainable params: 0

_________________________________________________________________


In [ ]:
loss, accuracy = brnn_model.evaluate(X_Test, Y_Test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

32/32 [==============================] - 8s 247ms/step - loss: 0.8522 - accuracy: 0.6970

Test Loss: 0.8522228598594666

Test Accuracy: 0.6970000267028809


In [ ]:
# Get the predicted labels
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = brnn_model.predict(X_Test)
y_pred = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded labels back to integers
y_true = np.argmax(Y_Test, axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.9133858267716536

Recall: 0.4522417153996101

F1-score: 0.604954367666232


## BiLSTM

In [ ]:
from tensorflow.keras.layers import  LSTM
blstm_model = Sequential()
blstm_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
blstm_model.add(Bidirectional(LSTM(100)))
blstm_model.add(Dense(2, activation='sigmoid'))

blstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
blstm_model.fit(X_Train, Y_Train, validation_data=(X_Test, Y_Test), batch_size = 128, epochs=10)

Epoch 1/10

63/63 [==============================] - 16s 200ms/step - loss: 0.6715 - accuracy: 0.6219 - val_loss: 0.6277 - val_accuracy: 0.7240

Epoch 2/10

63/63 [==============================] - 13s 203ms/step - loss: 0.4391 - accuracy: 0.8241 - val_loss: 0.4141 - val_accuracy: 0.8390

Epoch 3/10

63/63 [==============================] - 12s 196ms/step - loss: 0.3066 - accuracy: 0.8766 - val_loss: 0.3955 - val_accuracy: 0.8280

Epoch 4/10

63/63 [==============================] - 12s 192ms/step - loss: 0.2321 - accuracy: 0.9116 - val_loss: 0.3641 - val_accuracy: 0.8630

Epoch 5/10

63/63 [==============================] - 12s 191ms/step - loss: 0.1844 - accuracy: 0.9340 - val_loss: 0.3678 - val_accuracy: 0.8580

Epoch 6/10

63/63 [==============================] - 12s 195ms/step - loss: 0.1754 - accuracy: 0.9356 - val_loss: 0.3724 - val_accuracy: 0.8650

Epoch 7/10

63/63 [==============================] - 12s 189ms/step - loss: 0.1410 - accuracy: 0.9501 - val_loss: 0.4542 - val_acc

In [ ]:
blstm_model.summary()

Model: "sequential_4"

_________________________________________________________________

Layer (type)                 Output Shape              Param #   


module_wrapper_7 (ModuleWrap (None, 1806, 100)         400000    

_________________________________________________________________

module_wrapper_8 (ModuleWrap (None, 200)               160800    

_________________________________________________________________

dense_3 (Dense)              (None, 2)                 402       


Total params: 561,202

Trainable params: 561,202

Non-trainable params: 0

_________________________________________________________________


In [ ]:
loss, accuracy = blstm_model.evaluate(X_Test, Y_Test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

32/32 [==============================] - 1s 44ms/step - loss: 0.5590 - accuracy: 0.8620

Test Loss: 0.5590408444404602

Test Accuracy: 0.8619999885559082


In [ ]:
# Get the predicted labels
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = blstm_model.predict(X_Test)
y_pred = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded labels back to integers
y_true = np.argmax(Y_Test, axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.8427787934186471

Recall: 0.898635477582846

F1-score: 0.8698113207547171


## BiGRU

In [ ]:
from tensorflow.keras.layers import  GRU
bgru_model = Sequential()
bgru_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
bgru_model.add(Bidirectional(GRU(32)))
bgru_model.add(Dense(2, activation='sigmoid'))

bgru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
bgru_model.fit(X_Train, Y_Train, validation_data=(X_Test, Y_Test), batch_size = 128, epochs=10)

Epoch 1/10

63/63 [==============================] - 12s 119ms/step - loss: 0.6682 - accuracy: 0.5909 - val_loss: 0.5311 - val_accuracy: 0.7410

Epoch 2/10

63/63 [==============================] - 6s 99ms/step - loss: 0.4003 - accuracy: 0.8224 - val_loss: 0.3943 - val_accuracy: 0.8300

Epoch 3/10

63/63 [==============================] - 7s 108ms/step - loss: 0.2784 - accuracy: 0.8911 - val_loss: 0.3693 - val_accuracy: 0.8520

Epoch 4/10

63/63 [==============================] - 6s 100ms/step - loss: 0.2258 - accuracy: 0.9141 - val_loss: 0.3980 - val_accuracy: 0.8420

Epoch 5/10

63/63 [==============================] - 7s 109ms/step - loss: 0.1861 - accuracy: 0.9340 - val_loss: 0.4317 - val_accuracy: 0.8270

Epoch 6/10

63/63 [==============================] - 6s 100ms/step - loss: 0.1563 - accuracy: 0.9475 - val_loss: 0.4854 - val_accuracy: 0.8340

Epoch 7/10

63/63 [==============================] - 7s 108ms/step - loss: 0.1544 - accuracy: 0.9471 - val_loss: 0.4635 - val_accuracy: 

In [ ]:
loss, accuracy = bgru_model.evaluate(X_Test, Y_Test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

32/32 [==============================] - 1s 36ms/step - loss: 0.5256 - accuracy: 0.8260

Test Loss: 0.5256199836730957

Test Accuracy: 0.8259999752044678


In [ ]:
bgru_model.summary()

Model: "sequential_3"

_________________________________________________________________

Layer (type)                 Output Shape              Param #   


module_wrapper_5 (ModuleWrap (None, 1806, 100)         400000    

_________________________________________________________________

module_wrapper_6 (ModuleWrap (None, 200)               160800    

_________________________________________________________________

dense_2 (Dense)              (None, 2)                 402       


Total params: 561,202

Trainable params: 561,202

Non-trainable params: 0

_________________________________________________________________


In [ ]:
# Get the predicted labels
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = bgru_model.predict(X_Test)
y_pred = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded labels back to integers
y_true = np.argmax(Y_Test, axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.8466257668711656

Recall: 0.8070175438596491

F1-score: 0.8263473053892216


## Stacked RNN

In [ ]:
srnn_model = Sequential()
srnn_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
srnn_model.add(SimpleRNN(100, return_sequences=True))
srnn_model.add(SimpleRNN(100))
srnn_model.add(Dense(2, activation='sigmoid'))
srnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
srnn_model.fit(X_Train, Y_Train, validation_data=(X_Test, Y_Test), batch_size = 128, epochs=10)

Epoch 1/10

63/63 [==============================] - 269s 4s/step - loss: 0.7123 - accuracy: 0.5214 - val_loss: 0.6805 - val_accuracy: 0.5630

Epoch 2/10

63/63 [==============================] - 259s 4s/step - loss: 0.6307 - accuracy: 0.6574 - val_loss: 0.6162 - val_accuracy: 0.6560

Epoch 3/10

63/63 [==============================] - 259s 4s/step - loss: 0.5090 - accuracy: 0.7434 - val_loss: 0.6286 - val_accuracy: 0.6370

Epoch 4/10

63/63 [==============================] - 255s 4s/step - loss: 0.4828 - accuracy: 0.7751 - val_loss: 0.4782 - val_accuracy: 0.7740

Epoch 5/10

63/63 [==============================] - 256s 4s/step - loss: 0.3922 - accuracy: 0.8328 - val_loss: 0.4669 - val_accuracy: 0.7840

Epoch 6/10

63/63 [==============================] - 257s 4s/step - loss: 0.3175 - accuracy: 0.8687 - val_loss: 0.4696 - val_accuracy: 0.8060

Epoch 7/10

63/63 [==============================] - 257s 4s/step - loss: 0.2749 - accuracy: 0.8864 - val_loss: 0.5280 - val_accuracy: 0.7800


In [ ]:
srnn_model.summary()

Model: "sequential_7"

_________________________________________________________________

Layer (type)                 Output Shape              Param #   


module_wrapper_15 (ModuleWra (None, 1806, 100)         400000    

_________________________________________________________________

module_wrapper_16 (ModuleWra (None, 1806, 100)         20100     

_________________________________________________________________

module_wrapper_17 (ModuleWra (None, 100)               20100     

_________________________________________________________________

dense_6 (Dense)              (None, 2)                 202       


Total params: 440,402

Trainable params: 440,402

Non-trainable params: 0

_________________________________________________________________


In [ ]:
loss, accuracy = srnn_model.evaluate(X_Test, Y_Test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
# Get the predicted labels
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = srnn_model.predict(X_Test)
y_pred = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded labels back to integers
y_true = np.argmax(Y_Test, axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

32/32 [==============================] - 10s 303ms/step - loss: 0.6834 - accuracy: 0.7630

Test Loss: 0.6833871603012085

Test Accuracy: 0.7630000114440918

Precision: 0.7875

Recall: 0.7368421052631579

F1-score: 0.7613293051359517


## Stacked LSTM

In [ ]:
slstm_model = Sequential()
slstm_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
slstm_model.add(LSTM(100, return_sequences=True))
slstm_model.add(LSTM(100))
slstm_model.add(Dense(2, activation='sigmoid'))
slstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
slstm_model.fit(X_Train, Y_Train, validation_data=(X_Test, Y_Test), batch_size = 128, epochs=10)


Epoch 1/10

63/63 [==============================] - 20s 254ms/step - loss: 0.5499 - accuracy: 0.7036 - val_loss: 0.4290 - val_accuracy: 0.7930

Epoch 2/10

63/63 [==============================] - 13s 202ms/step - loss: 0.2868 - accuracy: 0.8865 - val_loss: 0.3327 - val_accuracy: 0.8580

Epoch 3/10

63/63 [==============================] - 14s 229ms/step - loss: 0.2036 - accuracy: 0.9227 - val_loss: 0.3633 - val_accuracy: 0.8720

Epoch 4/10

63/63 [==============================] - 12s 194ms/step - loss: 0.1455 - accuracy: 0.9474 - val_loss: 0.3710 - val_accuracy: 0.8500

Epoch 5/10

63/63 [==============================] - 12s 193ms/step - loss: 0.1175 - accuracy: 0.9576 - val_loss: 0.5106 - val_accuracy: 0.8500

Epoch 6/10

63/63 [==============================] - 13s 209ms/step - loss: 0.0751 - accuracy: 0.9737 - val_loss: 0.6201 - val_accuracy: 0.8530

Epoch 7/10

63/63 [==============================] - 13s 199ms/step - loss: 0.0522 - accuracy: 0.9826 - val_loss: 0.5837 - val_acc

In [ ]:
slstm_model.summary()

Model: "sequential_5"

_________________________________________________________________

Layer (type)                 Output Shape              Param #   


module_wrapper_9 (ModuleWrap (None, 1806, 100)         400000    

_________________________________________________________________

module_wrapper_10 (ModuleWra (None, 1806, 100)         80400     

_________________________________________________________________

module_wrapper_11 (ModuleWra (None, 100)               80400     

_________________________________________________________________

dense_4 (Dense)              (None, 2)                 202       


Total params: 561,002

Trainable params: 561,002

Non-trainable params: 0

_________________________________________________________________


In [ ]:
loss, accuracy = slstm_model.evaluate(X_Test, Y_Test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
# Get the predicted labels
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = slstm_model.predict(X_Test)
y_pred = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded labels back to integers
y_true = np.argmax(Y_Test, axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


32/32 [==============================] - 1s 45ms/step - loss: 0.7676 - accuracy: 0.8460

Test Loss: 0.7676162719726562

Test Accuracy: 0.8460000157356262

Precision: 0.8499025341130604

Recall: 0.8499025341130604

F1-score: 0.8499025341130604


In [ ]:
sgru_model = Sequential()
sgru_model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
sgru_model.add(GRU(100, return_sequences=True))
sgru_model.add(GRU(100))
sgru_model.add(Dense(2, activation='sigmoid'))
sgru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
sgru_model.fit(X_Train, Y_Train, validation_data=(X_Test, Y_Test), batch_size = 128, epochs=10)

Epoch 1/10

63/63 [==============================] - 15s 184ms/step - loss: 0.5979 - accuracy: 0.6554 - val_loss: 0.4316 - val_accuracy: 0.8080

Epoch 2/10

63/63 [==============================] - 11s 177ms/step - loss: 0.3968 - accuracy: 0.8225 - val_loss: 0.4611 - val_accuracy: 0.7880

Epoch 3/10

63/63 [==============================] - 15s 232ms/step - loss: 0.2793 - accuracy: 0.8864 - val_loss: 0.4265 - val_accuracy: 0.8230

Epoch 4/10

63/63 [==============================] - 11s 178ms/step - loss: 0.1932 - accuracy: 0.9273 - val_loss: 0.4360 - val_accuracy: 0.8540

Epoch 5/10

63/63 [==============================] - 11s 174ms/step - loss: 0.1431 - accuracy: 0.9470 - val_loss: 0.6317 - val_accuracy: 0.8360

Epoch 6/10

63/63 [==============================] - 11s 173ms/step - loss: 0.0864 - accuracy: 0.9720 - val_loss: 0.6682 - val_accuracy: 0.8140

Epoch 7/10

63/63 [==============================] - 11s 174ms/step - loss: 0.0685 - accuracy: 0.9780 - val_loss: 0.7361 - val_acc

In [ ]:
sgru_model.summary()

Model: "sequential_6"

_________________________________________________________________

Layer (type)                 Output Shape              Param #   


module_wrapper_12 (ModuleWra (None, 1806, 100)         400000    

_________________________________________________________________

module_wrapper_13 (ModuleWra (None, 1806, 100)         60600     

_________________________________________________________________

module_wrapper_14 (ModuleWra (None, 100)               60600     

_________________________________________________________________

dense_5 (Dense)              (None, 2)                 202       


Total params: 521,402

Trainable params: 521,402

Non-trainable params: 0

_________________________________________________________________


In [ ]:
loss, accuracy = sgru_model.evaluate(X_Test, Y_Test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
# Get the predicted labels
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = sgru_model.predict(X_Test)
y_pred = np.argmax(y_pred, axis=1)

# Convert the one-hot encoded labels back to integers
y_true = np.argmax(Y_Test, axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='binary')
recall = recall_score(y_true, y_pred, average='binary')
f1 = f1_score(y_true, y_pred, average='binary')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

32/32 [==============================] - 2s 73ms/step - loss: 1.1551 - accuracy: 0.7870

Test Loss: 1.1550843715667725

Test Accuracy: 0.7870000004768372

Precision: 0.8138075313807531

Recall: 0.7582846003898636

F1-score: 0.7850655903128154


In [1]:
from tabulate import tabulate

Table = [
    ["Accuracy Score:", 0.75,0.826,0.814,0.697,0.86199,0.826,0.763,0.846,0.787],
    ['Precision',0.7573,0.84809,0.881118,0.91338,0.84277879,0.846625,0.7875,0.8499,0.8138],
    ['recall',0.754385,0.80506822,0.73684,0.45224,0.898635,0.80701,0.73684,0.8499,0.75828],
    ['F1-score',0.755859,0.826,0.80254,0.604954,0.869811,0.826347,0.761329,0.8499,0.785066]
]

# Create header
head = ["Model", "RNN", "LSTM", "GRU", "Stacked_RNN", "Stacked_LSTM",
        "Stacked_GRU", "Bi_RNN", "Bi_LSTM", "Bi_GRU"]

# Display table
print(tabulate(Table, headers=head, tablefmt="grid"))

+-----------------+----------+----------+----------+---------------+----------------+---------------+----------+-----------+----------+
| Model           |      RNN |     LSTM |      GRU |   Stacked_RNN |   Stacked_LSTM |   Stacked_GRU |   Bi_RNN |   Bi_LSTM |   Bi_GRU |
+=================+==========+==========+==========+===============+================+===============+==========+===========+==========+
| Accuracy Score: | 0.75     | 0.826    | 0.814    |      0.697    |       0.86199  |      0.826    | 0.763    |    0.846  | 0.787    |
+-----------------+----------+----------+----------+---------------+----------------+---------------+----------+-----------+----------+
| Precision       | 0.7573   | 0.84809  | 0.881118 |      0.91338  |       0.842779 |      0.846625 | 0.7875   |    0.8499 | 0.8138   |
+-----------------+----------+----------+----------+---------------+----------------+---------------+----------+-----------+----------+
| recall          | 0.754385 | 0.805068 | 0.7368